In [88]:
import pyspark
import csv
import math
import pandas as pd
import numpy as np
from pyspark import SparkContext as sc
from pyspark import SparkConf
conf = SparkConf().setAppName("Term_Project_Group57").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)
conf = SparkConf().set("spark.default.parallelism", 4).set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

In [89]:
num_movie = 0
num_user = 610
data = []
movie = []
movie_list = {}
check_movie_id = []
cos = []
movie_dict = {} 
sim_dict = {}

In [90]:
def Preprocess() :
    global data, num_movie, check_movie_id
    data = pd.read_csv("ratings.csv", encoding="utf-8", names = ["User", "Movie", "Rating"])
    check_movie_id = sorted(list(set(data['Movie'].tolist())))
    num_movie = len(check_movie_id)
    for value in range(num_movie):
        movie_list.update({check_movie_id[value] :value})
    l = []
    with open( "ratings.csv", encoding="utf-8", newline = '' ) as file : # read file
        rows = csv.reader(file)
        for row in rows :
            l.append(row)
    pdata = sc.parallelize(l)
    return pdata

In [91]:
def change1(plist):
    plist = list(map(float, plist))
    # ((user, (movie, point))
    res = [int(plist[0]), [[int(plist[1]), plist[2]]]]      
    return res 

In [92]:
def change2(plist):
    plist = list(map(float, plist))
    # ((movie, (user, point))
    res = [int(plist[1]), [[int(plist[0]), plist[2]]]]      
    return res

In [93]:
def update_list1(plist) :
    # for about 9700 movies
    res_list = num_movie * [0]
    for movie_id, point in plist :
        idx = int(movie_id)
        res_list[movie_list[idx]] = point
    return tuple(res_list)

In [94]:
def update_list2(plist) :
    res_list = num_user * [0]
    for user_id, point in plist :
        idx = int(user_id)
        res_list[idx-1] = point
    return res_list

In [95]:
def Cal(plist) :
    count = 0
    for i in range(len(plist)) :
        if plist[i] != 0 :
            count += 1
    s = sum(plist) / count
    l = list(plist)
    for i in range(len(l)) :
        if l[i] != 0 :
            tmp = l[i] - s
            if tmp == 0 :
                l[i] = l[i] - s + 0.3
            else :
                l[i] = tmp
    temp = num_user * [0]
    num = 0
    for i in range(len(l)) :
        num = num + (l[i]*l[i])
    num = math.sqrt(num)
    return [num,l]

In [96]:
def Cos(plist) :
    u1 = plist[0][0]
    u2 = plist[0][1]
    if u1 != u2 :
        x = np.array(plist[1][1])
        y = np.array(plist[2][1])
        dot = np.dot(x, y)
        len1 = plist[1][0]
        len2 = plist[2][0]
        temp = len1 * len2
        sim = dot / temp
        if sim == 0 :
            sim += 0.1
        return ( u1, [[ u2, sim ]] )
    else :
        return ( u1, [[ u2, 1 ]])

In [97]:
def Cosine(plist) :
    global cos, movie_dict, sim_dict
    pdata = plist
    
    # [ ( user1, point1 ), ( user2, point2 ) ... ]
    plist = plist.map(change1).reduceByKey( lambda x, y : x + y ).mapValues(update_list1)
    
    # dictionary of user's point  ex : { user1 : 2 5 3 0, user2 : 3 0 2 5 ... }
    pmovie = pdata.map(change2).reduceByKey( lambda x, y : x + y ).mapValues(update_list2)
    
    # 計算每個user評分之length、計算每個評分扣掉mean ( 避免之後會divide zero狀況，所以算到0的話會一常數，本例是+0.3 )
    prate = plist.mapValues(Cal)
    
    # ( ( user1, user2 ), ( len1, ( point1 ) , len2, ( point2 ) ) ) 做user彼此之卡氏積
    pcar = prate.cartesian(prate).map(lambda x: (tuple((x[0][0], x[1][0])), x[0][1], x[1][1]))
    
    # ( user1, ( ( user2, sim(1,2) ) , ( user3, sim(1,3) ) ... ) ) (算出彼此之sim，如果sim是0一樣加常數 (+0.1))
    cos = pcar.map(Cos).reduceByKey(lambda a, b : a + b)

    movie_dict = pmovie.collectAsMap() 
    sim_dict = cos.collectAsMap()
    return plist

In [98]:
def Recommendation(plist) :
    user = plist[0]
    l = plist[1] # 原本user評分的list
    res = []
    for i in range(len(l)) :
        rate = l[i] 
        if rate == 0 :
            s = 0
            total_sim = 0
            point = movie_dict[check_movie_id[i]] #電影對所有user評分之狀況
            for j in range(len(point)) :
                sim = sim_dict[user] # user之間的sim
                s = s + ( point[j] * sim[j][1] )
                if point[j] != 0 : # 有其他user評過分才要加上他們的sim
                    total_sim = total_sim + sim[j][1]
            temp = s / total_sim
            tup = ( user, check_movie_id[i], temp )
            res.append(tup)
    return res

In [99]:
def Recommend(plist) :
    l = []
    res = plist.map(Recommendation)
    ans = res.collect()
    for i in ans :
        temp = sorted(i, key = lambda x : x[2])
        l.append(temp[-1])
    return l

In [100]:
def writefile(ans) :
    with open("final_output.txt", 'w') as file :
        for i in ans :
            User = i[0]
            movie = i[1]
            msg = 'Recommend User {} : Movie {}'.format(User, movie)
            file.writelines( msg + "\n" )

In [ ]:
pdata = Preprocess()
puser = Cosine(pdata)
ans = Recommend(puser)
res = sorted( ans, key = lambda x : x[0] )
writefile(res)

In [86]:
sc.stop()